### Installing packages

In [15]:
!pip install pandas
!pip install numpy
!pip install openpyxl


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
!pip install PyPDF2
!pip install pdfminer.six


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


### Scraping data from pdfs

In [44]:
import pandas as pd
import re
from pdfminer.high_level import extract_text, LAParams

Cleaning pdf

In [65]:
text = extract_text("oct22.pdf", laparams = LAParams(boxes_flow=None))
print(text)

TOMPKINS COUNTY SHERIFFS
OFFICE
Public Information Log

Domestic

Incident Address:  ENFIELD FALLS RD; T ITHACA

ITHACA NY 14850

Time Reported:  23:32:09

Time Occurred Between:  23:32:09 10/31/22 - 23:34:07 10/31/22

Traffic Offense

Incident Address:  BLOCK TRUMANSBURG RD; T ITHACA

ITHACA NY 14850

Time Reported:  23:19:50

Time Occurred Between:  23:19:50 10/31/22 - 23:19:50 10/31/22

Traffic Offense

Incident Address:  STATE ROUTE 13 & WILLOW AVE; C ITHACA

ITHACA NY 14850

Time Reported:  23:09:41

Time Occurred Between:  23:09:41 10/31/22 - 23:09:41 10/31/22

Traffic Stop

Alarm Police

Incident Address:  PLEASANT VALLEY RD; T GROTON
GROTON NY 13073

Time Reported:  23:04:56

Time Occurred Between:  23:04:56 10/31/22 - 23:06:10 10/31/22

Alarm activation

Traffic Offense

Incident Address:  STATE ROUTE 13 N & N TRIPHAMMER RD; V LANSING

ITHACA NY 14850

Time Reported:  22:55:28

Time Occurred Between:  22:55:28 10/31/22 - 22:55:28 10/31/22

11/01/22

Public Information Log

Tr

In [72]:
ts = text.split('\n')
ts = [i for i in ts if i != '']

f = open('clean.txt', 'w')
for i in ts:
    # remove hex characters
    line = re.sub(r'[^\x00-\x7f]',r'', i) 

    # heuristical skipping
    if i in ['TOMPKINS COUNTY SHERIFFS','OFFICE', 'Public Information Log', 'Public Information Log']:
        continue
    if i[:4] == 'Page':
        continue
    if i[0].isdigit():
        continue
            
    print(line, file=f)

f.close()

Need to process some things manually after - very first reason doesn't appear for some reason

In [83]:
reasons = []
incident_address = []
time_reported = []
time_occured = []
comments = []

if_last_is_incident = False
if_last_time_occured = True

tot = ""
prev = ""
with open('clean.txt', 'r') as f:
    for ind, line in enumerate(f.readlines()):
        line = line.strip()
        
        first_colon = line.find(':')

        # heuristic adding to 
        if first_colon != -1:
            before = line[:first_colon]
            after = line[first_colon + 1:]
            if before == 'Incident Address':
                
                # append previous comments if not start as well as next reason
                if ind != 1:
                    comments.append(tot)
                reasons.append(prev)
                tot = ""
                prev = ""
                if_last_time_occured = False


                incident_address.append(after)

                if_last_is_incident = True

            elif before == 'Time Reported':
                if_last_is_incident = False
                time_reported.append(after)
            elif before == 'Time Occurred Between':
                time_occured.append(after)
                if_last_time_occured = True
        else:
            if if_last_is_incident:
                # Add to address
                incident_address[-1] += ' ' + line
            elif if_last_time_occured:
                tot += " " + prev
                prev = line
            # if first line
            elif ind == 0:
                prev = line

            else:
                pass
    comments.append(tot)
len(incident_address), len(time_reported), len(time_occured), len(reasons), len(comments)

(1841, 1841, 1841, 1841, 1841)

In [84]:
df = pd.DataFrame.from_dict({
    'reasons' : reasons,
    'time_occured' : time_occured,
    'time_reported' : time_reported,
    'comments' : comments,
    'incident_address' : incident_address
})

,reasons,time_occured,time_reported,comments,incident_address
0,Domestic,23:32:09 10/31/22 - 23:34:07 10/31/22,23:32:09,,ENFIELD FALLS RD; T ITHACA ITHACA NY 14850
1,Traffic Offense,23:19:50 10/31/22 - 23:19:50 10/31/22,23:19:50,,BLOCK TRUMANSBURG RD; T ITHACA ITHACA NY 14850
2,Traffic Offense,23:09:41 10/31/22 - 23:09:41 10/31/22,23:09:41,Traffic Stop,STATE ROUTE 13 & WILLOW AVE; C ITHACA ITHACA N...
3,Alarm Police,23:04:56 10/31/22 - 23:06:10 10/31/22,23:04:56,Alarm activation,PLEASANT VALLEY RD; T GROTON GROTON NY 13073
4,Traffic Offense,22:55:28 10/31/22 - 22:55:28 10/31/22,22:55:28,,STATE ROUTE 13 N & N TRIPHAMMER RD; V LANSING ...
...,...,...,...,...,...
1836,Traffic Offense,00:37:21 10/01/22 - 00:37:21 10/01/22,00:37:21,Traffic Stop,ELMIRA RD; SMILEYS CITGO; T NEWFIELD NEWFIELD ...
1837,Property Check,00:37:06 10/01/22 - 00:37:06 10/01/22,00:37:06,Property Check.,SILL RD; T LANSING LOCKE NY 13092
1838,Special Detail,00:30:58 10/01/22 - 00:30:58 10/01/22,00:30:58,Detail area checked. RPS/1234,LANSINGVILLE RD; T LANSING LANSING NY 14882
1839,Special Detail,00:26:09 10/01/22 - 00:26:09 10/01/22,00:26:09,Area check. RPS/1234,EMMONS RD; T LANSING LANSING NY 14882


Cleaning csvs

In [19]:
f = pd.read_csv('oct22accidents.csv')

In [23]:
f.columns = f.iloc[1]

In [26]:
f.dropna(how='all', axis=1, inplace=True)

In [29]:
f.drop([0,1], inplace=True)
f.to_csv('clean.csv')